# 🧩 Optimización SQL y Particionado de Datos

Objetivo: entender y aplicar técnicas de optimización de consultas (índices, estadísticas, JOIN strategies) y estrategias de particionado (bases relacionales y data lakes) para mejorar desempeño y costos.

- Duración: 90–120 min
- Dificultad: Media
- Prerrequisitos: SQL intermedio, nociones de modelado y almacenamiento columnar

## 1. EXPLAIN / EXPLAIN ANALYZE (demo con SQLite)

In [ ]:
import sqlite3, pandas as pd
conn = sqlite3.connect(':memory:')
cur = conn.cursor()
cur.executescript('''
CREATE TABLE ventas (
 id INTEGER PRIMARY KEY, fecha TEXT, cliente_id INT, producto_id INT, cantidad INT, precio REAL, total REAL
);
CREATE INDEX idx_ventas_cliente_fecha ON ventas (cliente_id, fecha);
''')
import random, datetime as dt
rows = [ (i, str(dt.date(2025,10,(i%28)+1)), random.randint(1,200), random.randint(100,110), random.randint(1,5), round(random.uniform(5,200),2)) for i in range(1,5001) ]
rows = [ (*r, r[4]*r[5]) for r in rows ]
cur.executemany('INSERT INTO ventas (id, fecha, cliente_id, producto_id, cantidad, precio, total) VALUES (?,?,?,?,?,?,?)', rows)
conn.commit()
list(cur.execute('EXPLAIN QUERY PLAN SELECT * FROM ventas WHERE cliente_id=10 AND fecha>="2025-10-10"').fetchall())[:5]

### 1.1 Impacto de índices compuestos y selectividad

- Use índices compuestos en el orden de los predicados más selectivos.
- Evite funciones sobre columnas indexadas en filtros (rompe el índice).
- Mantenga estadísticas actualizadas (ANALYZE).

## 2. Particionado en PostgreSQL (DDL de referencia)

In [ ]:
ddl = '''
-- Tabla particionada por rango de fecha
CREATE TABLE ventas_part (
  id BIGSERIAL PRIMARY KEY,
  fecha DATE NOT NULL,
  cliente_id INT,
  producto_id INT,
  cantidad INT,
  total NUMERIC(12,2)
) PARTITION BY RANGE (fecha);
-- Particiones mensuales
CREATE TABLE ventas_2025_10 PARTITION OF ventas_part FOR VALUES FROM ('2025-10-01') TO ('2025-11-01');
CREATE INDEX ON ventas_2025_10 (fecha, cliente_id);
ANALYZE ventas_2025_10;
-- Consulta con pruning de particiones
EXPLAIN ANALYZE SELECT SUM(total) FROM ventas_part WHERE fecha >= '2025-10-10' AND fecha < '2025-10-20';
'''
print(ddl)

## 3. Data Lakes: particionado y pruning (Parquet)

- Particionar por columnas de alta cardinalidad temporal (por ejemplo, año/mes/día).
- Ajustar tamaño de archivos (128–512 MB) para evitar small files problem.
- Pruning: motores como Spark/Athena/Trino escanean solo particiones necesarias.
- Usa formato columnar (Parquet) y compresión (Snappy/ZSTD).

## 4. Window functions y agregaciones eficientes

In [ ]:
import pandas as pd
df = pd.read_sql_query('SELECT fecha, cliente_id, total FROM ventas', conn)
df['fecha'] = pd.to_datetime(df['fecha'])
df = df.sort_values(['cliente_id','fecha'])
df['acumulado_cliente'] = df.groupby('cliente_id')['total'].cumsum()
df.head()

## 5. Buenas prácticas

- Evitar SELECT * en producción; proyectar solo columnas necesarias.
- Usar límites de tiempo y paginación en consultas intensivas.
- Mantener índices y particiones según patrones de acceso actuales.
- Monitorear planes y tiempos con muestras periódicas; automatizar regresiones.